In [ ]:
"""
    Pappus' creativity is due to him (implicitly) distinguishing between numerical identity and conceptual identity
    in step 1 and applying this distinction to the triangle ABC to take advantage of Euclid I.4.
"""

!python --version

# install and import packages
!pip install rdflib

import glob
import json
import os
import rdflib
import requests
import typing

# !pwd

# PROOFS OF EUCLID I.5

In [ ]:
# import json
# import requests

class Row:
    """ Represents a single row of SPARQL query results. """
    def __init__(self, data):
        self.__dict__.update(data)
    """
    self.__dict__.update(data): 
    This line uses the update method 
    on the __dict__ attribute of the 
    object (self). The __dict__ attribute 
    is a special dictionary that stores 
    all the attributes (key-value pairs) 
    associated with the object. 
    By calling update on it with the 
    data dictionary, the __init__ method 
    effectively assigns all the key-value 
    pairs from the data dictionary 
    as attributes of the newly 
    created object.
    """

class SparqlQueryResults():
    """ Represents the results of a SPARQL query. """

    # initialize objects
    def __init__(self,
                sparql_query,
                rdfox_server:str = "http://localhost:12110",
                datastore:str = "/datastores/foom_pappus/sparql",
                format_response:str = "application/sparql-results+json"):
        self.sparql_query = sparql_query
        self.rdfox_server = rdfox_server
        self.datastore = datastore
        self.format_response = format_response
        self.response_text = self.query_ontology()
        self.response_dictionary = self.response_to_dictionary()
        self.rows = self.create_rows()
        self.index = 0  # For the iterator in the __next__ method

    # submit sparql query and retrieve results
    def query_ontology(self) -> str:
        """ Submit SPARQL query and retrieve results. """
        # define the headers of the request
        headers = {
            "Accept": self.format_response
        }
        
        # query the datastore at the rdfox server
        response = requests.get(f"{self.rdfox_server}{self.datastore}",
                                params={"query": self.sparql_query},
                                headers=headers)
        
        # check if the REST endpoint returns an unexpected status code
        response.raise_for_status()
        
        # if the REST endpoint returns an expected response, return the text of the response
        return response.text

    # transform the text response to the ontology query
    # into a python dictionary and
    # extract the relevant information about
    # headers and values
    def response_to_dictionary(self) -> dict:
        """ Transform the SPARQL query response into a dictionary. """
        
        # transform the text response to the ontology query
        # into a json object
        response_data = json.loads(self.response_text)

        # select the headers for the new dictionary of data
        headers:list = response_data["head"]["vars"]

        # transform the json response into a dictionary
        # with headers and values
        response_dictionary:dict = {header: [] for header in headers}
        for binding in response_data["results"]["bindings"]:
            for header in headers:
                response_dictionary[header].append(binding[header]["value"])
                
        return response_dictionary

    # create rows of results
    def create_rows(self) -> list:
        """ Create Row objects for each result. """

        # list to append rows
        rows:list = []

        # find header names and number of rows to add to the list of rows
        headers:list = list(self.response_dictionary.keys())
        number_of_rows= len(self.response_dictionary[headers[0]])

        # add rows to the list of rows
        for index in range(number_of_rows):
            row_data = {header:self.response_dictionary[header][index]
                       for header in headers}
            rows.append(Row(row_data))

        return rows

    # make the object iterable using
    # the __iter__ functions and
    # the __next_ function
    def __iter__(self):
        return self

    def __next__(self):
        """ Iterate through the rows of the results. """
        if self.index < len(self.rows):
            result = self.rows[self.index]
            self.index += 1
            return result
        else:
            self.index = 0  # Reset for future iteration
            raise StopIteration

In [ ]:
sparql_text_0:str = """
        select distinct 
        ?iri
        ?label
        where {
        ?s ?iri ?o .
        ?iri rdfs:label ?label
        }
        order by ?label
        limit 2
    """
# Example usage
sparql_query_results = SparqlQueryResults(sparql_text_0)
for row in sparql_query_results:
    print(row.iri)

In [ ]:
#### SPARQL QUERIES
# find all proofs
def sparql_query_find_proofs() -> str:
    sparql_query:str = """
        SELECT DISTINCT 
            ?proof_iri
            ?proof_label
        WHERE {
            ?proof_iri 
                a <http://www.foom.com/core/Proof> ;
                rdfs:label ?proof_label .
        }
    """
    return sparql_query

sparql_query_result_proofs = SparqlQueryResults(sparql_query_find_proofs())

for row in sparql_query_result_proofs:
    print(row.proof_iri)

In [ ]:
type(sparql_query_result_proofs)

In [ ]:
# 0. select a proof
# find all proofs


# 1. statement to prove


# 2. conceptual items connected to that statement
# 3. apply heuristics
# 4. compare step with results of heuristics
# 5. repeat for each step



In [ ]:
#### SPARQL QUERIES

# find all proofs
def sparql_query_find_proofs():
    sparql_query:str = """
        SELECT DISTINCT 
            ?proof_iri
            ?proof_label
        WHERE {
            ?proof_iri 
                a <http://www.foom.com/core/Proof> ;
                rdfs:label ?proof_label .
        }
    """
    return sparql_query

# for every proof, find all the steps in that proof
def sparql_query_find_steps_in_proof(given_proof_iri:str):
    sparql_query:str = f"""
    SELECT DISTINCT
        ?proof_iri
        ?proof_step_iri
        ?proof_step_label
        ?antecendent_proof_step_iri
    WHERE {{
        ?proof_iri 
            a <http://www.foom.com/core/Proof> .
        ?proof_step_iri
            a <http://www.foom.com/core/Proof_step> ;
            rdfs:label ?proof_step_label ;
            <http://www.foom.com/core/inProof> {given_proof_iri} .
        OPTIONAL {{
            ?proof_step_iri <http://www.foom.com/core/hasAntecedent> ?antecendent_proof_step_iri .
        }}
    }}
    """
    return sparql_query

# find all the individuals and properties used in the proof steps of a given proof
# group by proof step
def sparql_query_find_individuals_and_properties_of_proof_steps_of_given_proof(given_proof_iri:str):
    sparql_query:str = f"""
    SELECT DISTINCT
        ?proof_iri
        ?proof_step_iri
    WHERE {{
        ?proof_iri 
            a <http://www.foom.com/core/Proof> .
        ?proof_step_iri
            a <http://www.foom.com/core/Proof_step> ;
            <http://www.foom.com/core/inProof> {given_proof_iri} .
        
    }}
    """
    return sparql_query

# find exactly all the properties used in some proof step
def sparql_query_find_properties_proof_steps():
    sparql_query:str = """
        SELECT DISTINCT 
            ?p 
            ?p_label
            ?p_type
        WHERE {
            ?proof_step_iri 
                a <http://www.foom.com/core/Proof_step> ;
                ?p ?o .
            ?p 
                rdfs:label ?p_label ;
                a ?p_type .
        }
    """
    return sparql_query
    

# for every step, find the triple attached to it and the items to which the step directly refers

